# Working with the AMD GPU tools

The GPU is another accelerator with its own set of tools and utilities to interact with. In this notebook we'll walk through how to use the utilities that get shipped with the ROCm runtime to inspect our GPU status and utilization.

## Goals

* Get to know the ROCm stack drivers
* Learn to use `rocm-smi` and `amdgpu_top`

## GPU Stack

There are 3 main entrypoints to interacting with AMD GPUs on the Ryzen AI platform. There is the ROCm runtime, the HIP SDK and the Vulkan API. Each of these has its own benefits. In this section we will explore each of them.

ROCm is a stack of software including the GPU driver, runtime, utilities and optimized libraries for AMD GPUs. 

The HIP SDK can be considered a lower level component of ROCm and consists of a compiler toolchain for code running on the GPU written in the HIP kernel language.

Vulkan is an open standard for high performance graphics designed for modern multicore CPU systems and GPUs. Vulkan enables very low level access to GPU hardware. Although built as a graphics API, Vulkan offers general purpose computation through "compute shaders". Shaders are small programs written in a domain specific languages like glsl that run on GPU compute units.

## Utilities

### ROCm SMI

To interact with an AMD GPU we may use a variety of utilities. We will start with the `rocm-smi` (ROCm system management interface) tool. This tool is useful for:

* Confirming the GPU driver is loaded correctly
* Enumerating GPUs
* Examining and configuring GPUs

Let's explore the use of `rocm-smi`. First, execute `rocm-smi` without any parameters:

In [1]:
! rocm-smi



======================================== ROCm System Management Interface ========================================
================================================== Concise Info ==================================================
Device  Node  IDs              Temp    Power     Partitions          SCLK  MCLK  Fan  Perf  PwrCap  VRAM%  GPU%  
              (DID,     GUID)  (Edge)  (Socket)  (Mem, Compute, ID)                                              
0       1     0x1586,   14139  38.0°C  12.093W   N/A, N/A, 0         N/A   N/A   0%   auto  N/A     1%     1%    
============================================== End of ROCm SMI Log ===============================================


Here we see we have 1 GPU present, indicating that the GPU driver is loaded, along with other information about temperature, power consumption, memory allocated and compute utilization. These values can also be queried directly for example for utilization:

In [2]:
! rocm-smi -u



============================ ROCm System Management Interface ============================
=================================== % time GPU is busy ===================================
GPU[0]		: GPU use (%): 1
================================== End of ROCm SMI Log ===================================


A full listing of parameters can be seen by passing the -h parameters. Modify the cell below to use a parameter of your choice.

In [3]:
! rocm-smi -h

usage: rocm-smi [-h] [-V] [-d DEVICE [DEVICE ...]] [--alldevices] [--showhw] [-a] [-i] [-v] [-e [EVENT ...]]
                [--showdriverversion] [--showtempgraph] [--showfwinfo [BLOCK ...]] [--showmclkrange] [--showmemvendor]
                [--showsclkrange] [--showproductname] [--showserial] [--showuniqueid] [--showvoltagerange] [--showbus]
                [--showpagesinfo] [--showpendingpages] [--showretiredpages] [--showunreservablepages] [-f] [-P] [-t]
                [-u] [--showmemuse] [--showvoltage] [-b] [-c] [-g] [-l] [-M] [-m] [-o] [-p] [-S] [-s]
                [--showmeminfo TYPE [TYPE ...]] [--showpids [VERBOSE]] [--showpidgpus [SHOWPIDGPUS ...]]
                [--showreplaycount] [--showrasinfo [SHOWRASINFO ...]] [--showvc] [--showxgmierr] [--showtopo]
                [--showtopoaccess] [--showtopoweight] [--showtopohops] [--showtopotype] [--showtoponuma]
                [--showenergycounter] [--shownodesbw] [--showcomputepartition] [--showmemorypartition] [--showmetr

## amdgpu_top

`amdgpu_top` is another utility providing realtime statistics of AMD GPUs including utilization, memory consumption and specifics of individual processes. In a separate terminal, launch `amdgpu_top`. It can be closed by typing `q`. 

![](images/new_terminal.png)

`amdgpu_top`

## Vulkan

Running the `vulkaninfo` command enumerates the features of any GPUs present. Running the command below gives a summary of your system and validates that the vulkan environment is correctly setup. If running on the AMD AUP cloud, this command will show the presence of a  Strix Halo GPU (gfx1151).

In [4]:
! vulkaninfo --summary

error: XDG_RUNTIME_DIR is invalid or not set in the environment.
VULKANINFO

Vulkan Instance Version: 1.4.313


Instance Extensions: count = 24
-------------------------------
VK_EXT_acquire_drm_display             : extension revision 1
VK_EXT_acquire_xlib_display            : extension revision 1
VK_EXT_debug_report                    : extension revision 10
VK_EXT_debug_utils                     : extension revision 2
VK_EXT_direct_mode_display             : extension revision 1
VK_EXT_display_surface_counter         : extension revision 1
VK_EXT_headless_surface                : extension revision 1
VK_EXT_surface_maintenance1            : extension revision 1
VK_EXT_swapchain_colorspace            : extension revision 5
VK_KHR_device_group_creation           : extension revision 1
VK_KHR_display                         : extension revision 23
VK_KHR_external_fence_capabilities     : extension revision 1
VK_KHR_external_memory_capabilities    : extension revision 1
VK_KHR_external_

## HIP

In this example you can use HIP to query properties of the GPU. `hipcc` is the HIP compiler. We pass a file called `test.hip` to the compiler which outputs `test` as an executable output. In this example we detect the name and total memory available to the GPU.


In [5]:
! hipcc hip/test.hip -o hip/test && ./hip/test

Hello ROSCON
GPU detected: AMD Radeon Graphics
Total memory: 32768 MB


You can inspect the code by running the cell below

In [6]:
! cat hip/test.hip

#include <iostream>
#include <hip/hip_runtime.h>

int main (void) {
    // Detect platform
    hipDeviceProp_t dProperties;
    auto err = hipGetDeviceProperties(&dProperties, 0);
    
    if(err!=HIP_SUCCESS) {
        return -1;
    }
    std::cout << "Hello ROSCON" << std::endl;
    std::cout << "GPU detected: " << dProperties.name << std::endl;
    std::cout << "Total memory: " << dProperties.totalGlobalMem /1024 /1024 << " MB" << std::endl;
    return 0;
}

## Key Takeaways

In this notebook you have learned

- How to use `rocm-smi` and `amdgpu_top` to monitor GPU status.
- How to use `vulkaninfo` to verify your Vulkan installation.
- How to compile a basic HIP program.

## Cleanup

Make sure to `ctrl+c` and close the terminals before moving onto the next notebook.

## References

* [Vulkan](https://www.vulkan.org/)
* [rocm-smi](https://rocm.docs.amd.com/projects/rocm_smi_lib/en/latest/)
* [amdgpu_top](https://github.com/Umio-Yasuno/amdgpu_top)

**Continue to:** [2_running_llms.ipynb](2_running_llms.ipynb)


---
Copyright© 2025 AMD, Inc SPDX-License-Identifier: MIT